In [1]:
from tensorflow import keras 
import pandas as pd
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import copy
import numpy as np

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
model = keras.models.load_model('C:/Users/Moh.Massoud/ML/DrowsinessDetectionAlexNet.h5')

In [4]:
import cv2
import dlib

cap = cv2.VideoCapture(1) # Check if the webcam is opened correctLy 
if not cap.isOpened():
    cap = cv2.VideoCapture('http://192.168.1.4:4747/video') 
if not cap.isOpened(): 
    raise IOError("Cannot open webcam")  

#cap.set(cv2.CAP_PROP_FPS, 50)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/Moh.Massoud/ML/shape_predictor_68_face_landmarks.dat")
RIGHT_EYE_POINTS = list(range(36, 42))  
LEFT_EYE_POINTS = list(range(42, 48))  
status=["closed","opened"]
font = cv2.FONT_HERSHEY_SIMPLEX
X=[0,0]
Y=[0,0]
engage=0
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    predictions=[]
    roi=[]
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        landmarks = predictor(gray, face)
        mx_x=-np.inf
        mn_x=np.inf
        mx_y=-np.inf
        mn_y=np.inf
        for n in RIGHT_EYE_POINTS:
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            mx_x=max(mx_x,x)
            mn_x=min(mn_x,x)
            mx_y=max(mx_y,y)
            mn_y=min(mn_y,y)
            #cv2.circle(frame, (x, y), 4, (0, 255, 0), -1)
        x=mn_x-10
        y=mn_y-10
        X[0]=x
        Y[0]=y
        w=(mx_x-mn_x)+15
        h=(mx_y-mn_y)+15
        cv2.rectangle(frame, (x, y), (x+w,y+h), (0, 255, 0), 2)
        roi.append(cv2.resize(cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY), (224,224)))
        mx_x=-np.inf
        mn_x=np.inf
        mx_y=-np.inf
        mn_y=np.inf
        for n in LEFT_EYE_POINTS:
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            mx_x=max(mx_x,x)
            mn_x=min(mn_x,x)
            mx_y=max(mx_y,y)
            mn_y=min(mn_y,y)
        x=mn_x-10
        y=mn_y-10
        X[1]=x
        Y[1]=y
        w=(mx_x-mn_x)+15
        h=(mx_y-mn_y)+15
        cv2.rectangle(frame, (x, y), (x+w,y+h), (0, 255, 0), 2)
        roi.append(cv2.resize(cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY), (224,224)))
    if(len(roi)>0):
        test_images = np.expand_dims(roi, axis=3)
        predictions = model.predict(test_images) 
    temp=0
    for i in range(len(predictions)):
        predict=np.argmax(predictions[i])
        temp+= not predict
        cv2.putText(frame, status[predict], (X[i], Y[i]-20), font, 0.6, (0, 0, 255), 1, cv2.LINE_4)
    if temp:
        engage+=1
    else :
        engage=0
    if engage>30:
        cv2.putText(frame, "Sleeping", (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.putText(frame, f'{engage} : {temp}', (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.imshow("Drowsiness detection", frame)
    if cv2.waitKey(2) & 0xFF == ord('q'): 
        break 
cap. release() 
cv2.destroyAllWindows() 

In [5]:
cap. release() 
cv2.destroyAllWindows() 